In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment
import librosa
import soundfile as sf

def vol():
  while True:
    volume = int(input('Input volume (-10000 until 100): '))
    if volume >= -10000 and volume <= 100:
      break
    print("Volume must be -10000 until 100")

  while True:
    file_name = input('Input filename: ')
    if file_name.endswith('.wav'):
      break
    print("File name must ends with .wav")

  audio = AudioSegment.from_wav('speech.wav')
  audio = audio + volume
  # Export pakai soundfile
  audio_data = audio.get_array_of_samples()
  sf.write(file_name, audio_data, audio.frame_rate)

def pitch():
  while True:
    pitch = int(input('Input pitch (-10 until 10): '))
    if pitch >= -10 and pitch <= 10:
      break
    print("Pitch must be -10 until 10")

  while True:
    file_name = input('Input filename: ')
    if file_name.endswith('.wav'):
      break
    print("File name must ends with .wav")

  audio_data, sample_rate = librosa.load('speech.wav')
  pitch_audio = librosa.effects.pitch_shift(audio_data, sr=sample_rate, n_steps=pitch)
  # Export
  sf.write(file_name, pitch_audio, sample_rate)

def fade_in():
  while True:
    duration = int(input('Input duration: '))
    if duration > 0:
      break
    print("Duration must be more than 0")

  while True:
    file_name = input('Input filename: ')
    if file_name.endswith('.wav'):
      break
    print("File name must ends with .wav")

  audio = AudioSegment.from_wav('speech.wav')
  audio = audio.fade_in(duration * 1000)
  # Export pakai soundfile
  audio_data = audio.get_array_of_samples()
  sf.write(file_name, audio_data, audio.frame_rate)

def fade_out():
  while True:
    duration = int(input('Input duration: '))
    if duration > 0:
      break
    print("Duration must be more than 0")

  while True:
    file_name = input('Input filename: ')
    if file_name.endswith('.wav'):
      break
    print("File name must ends with .wav")

  audio = AudioSegment.from_wav('speech.wav')
  audio = audio.fade_out(duration * 1000)
  # Export pakai soundfile
  audio_data = audio.get_array_of_samples()
  sf.write(file_name, audio_data, audio.frame_rate)

In [ ]:
import torch
import torchaudio

class GreedyCTCDecoder(torch.nn.Module):
  def __init__(self, labels):
    super().__init__()
    self.labels = labels
    self.blank = 0

  def forward(self, emission: torch.Tensor) -> str:
    indices = torch.argmax(emission, dim=-1)
    indices = torch.unique_consecutive(indices, dim=-1)
    indices = [i for i in indices if i != self.blank]
    labels = [self.labels[i].replace('|', ' ') for i in indices]

    return ''.join(labels)

def transcript():
  bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
  model = bundle.get_model()
  waveform, sample_rate = torchaudio.load('speech.wav')

  if sample_rate != bundle.sample_rate:
    waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

  with torch.inference_mode():
    emission, _ = model(waveform)

  decoder = GreedyCTCDecoder(bundle.get_labels())
  text = decoder(emission[0])
  print(text)

In [ ]:
def menu():
    print('Program')
    print('1. Augmentation')
    print('2. Prediction')
    print('0. Exit')
    choice = input('Choose menu: ')
    return choice

def submenu():
    print('Augmentation')
    print('1. Volume Up/Down')
    print('2. Pitch')
    print('3. Add Fade In')
    print('4. Add Fade Out')
    print('0. Back')
    subchoice = input('Choose menu: ')
    return subchoice

while True:
    choice = menu()
    if choice == '1':
        while True:
            subchoice = submenu()
            if subchoice == '1':
                vol()
            elif subchoice == '2':
                pitch()
            elif subchoice == '3':
                fade_in()
            elif subchoice == '4':
                fade_out()
            elif subchoice == '0':
                break
            else:
                print('Input only 0-4')
    elif choice == '2':
      transcript()
    elif choice == '0':
      print('Thank you')
      break
    else:
      print('Input 0-3 only')

Program
1. Augmentation
2. Prediction
0. Exit
Choose menu: 2
THE PUBLIC ARE ENTREATED TO BEAR IN MIND THAT THIRTEEN YEARS HAVE PASSED SINCE IT WAS FINISHED MANY MORE SINCE IT WAS BEGUN AND THAT DURING THAT PERIOD PLACES MANNERS BOOKS AND OPINIONS HAVE UNDERGONE CONSIDERABLE CHANGES 
Program
1. Augmentation
2. Prediction
0. Exit
Choose menu: 0
Thank you
